In [1]:
# play audio in ipython notebook
try:
    import winsound
except ImportError:
    !pip install winsound
    import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [2]:
import numpy as np
import time
import requests
import os
import sys
import pickle
from lxml import html
import numpy as np
import urllib.request
import face_recognition
import cv2
import matplotlib.pyplot as plt
import pytesseract
from googlesearch import search
from PIL import Image, ImageStat
import time
import logging


import warnings
warnings.filterwarnings("ignore")
from deepface import DeepFace
deepface_logger = logging.getLogger("deepface")
deepface_logger.setLevel(level=logging.DEBUG)

import tqdm
class _TQDM(tqdm.tqdm):
    def __init__(self, *argv, **kwargs):
        kwargs['disable'] = True
        if kwargs.get('disable_override', 'def') != 'def':
            kwargs['disable'] = kwargs['disable_override']
        super().__init__(*argv, **kwargs)
tqdm.tqdm = _TQDM
tt = time.time
st = tt()

Using TensorFlow backend.


time: 13.6 s


In [3]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    print(bright)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

def change_img_name(strn):
    temp = int(int(strn.split("/")[-1][:-4])/100)
    return temp

def stopPrint(func, *args, **kwargs):
    with open(os.devnull,"w") as devNull:
        original = sys.stdout
        sys.stdout = devNull
        res = func(*args, **kwargs)
        sys.stdout = original 
    return res

time: 15.9 ms


In [4]:
base_dir = 'C:\\Users\\YASH\\nptel_face_recognition'
data_dir = os.path.join(base_dir, 'data')
results_dir = os.path.join(base_dir, "results")
os.makedirs(results_dir, exist_ok = True)

ver = '3'
known_faces_dir = os.path.join(data_dir, 'known_faces'+ver)
full_vedios_dir = os.path.join(data_dir, 'full_vedio')
pkl_files_dir = os.path.join(data_dir, "pkl_files")

folder_name_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict.pkl"))

images_dir_list = [x[0] for x in os.walk(known_faces_dir)][1:]
known_face_names = [folder_name_2_prof_name_dict[int(w.split('\\')[-1])] for w in images_dir_list]

known_face_img_path_nested_list = []
for image_dir in images_dir_list:
    known_images_path_list = get_extension_files(image_dir, ext='.jpg') 
    known_face_img_path_nested_list.append(known_images_path_list)

# winsound.Beep(freq, duration)

time: 32.9 ms


In [5]:
# save_as_pickle(os.path.join(pkl_files_dir, f"known_face_names"+ver+".pkl"), known_face_names)
# save_as_pickle(os.path.join(pkl_files_dir, f"known_images_path_list"+ver+".pkl"), known_images_path_list)

time: 5.98 ms


In [ ]:
tracebacks_dir = os.path.join(data_dir, "tracebacks")
os.makedirs(tracebacks_dir, exist_ok = True)

pkl_files_dir = os.path.join(data_dir, "pkl_files")
sgdf_known_faces_dir = os.path.join(data_dir, 'known_faces_sgdf')

nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
playlist_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"))

known_face_names = load_from_pickle(os.path.join(pkl_files_dir, f"known_face_names"+ver+".pkl"))
known_images_path_list = load_from_pickle(os.path.join(pkl_files_dir, f"known_images_path_list"+ver+".pkl"))

deep_face_metric = "euclidean"
best_backend = "opencv"
temp_img_path = "../data/img.jpg"
serengil_deep_face_models_list = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "Dlib"]

# start after labeled faces
# TODO: customize vedio start idx as all vedios dont take images from the first 2 vedios
vedio_start_idx = 2
# treshold for face verification
threshold = 0.25
max_pred_cnt = 5
num_labeled_faces = 2

num_channels = 2
num_playlists = 5
num_vedios = 5
    
# num_channels = 5
# num_playlists = 5
# num_vedios = 5

best_model_list = []
# for nptel_channel_name in nptel_channel_name_list:
for nptel_channel_name in nptel_channel_name_list[1:num_channels]:
    print(f'\n\n\n\nfor nptel_channel_name {nptel_channel_name}')
    chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)
    facef_playlist_names_list = load_from_pickle(os.path.join(chanel_pkl_files_dir, "facef_playlist_names_list.pkl"))

    results_chanel_dir = os.path.join(results_dir, nptel_channel_name)
    chanel_vedio_dir = os.path.join(full_vedios_dir, nptel_channel_name)
    paylist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]
    paylist_name_list = [x.split('\\')[-1] for x in paylist_path_list] 
    
    vedio_ext = '.mp4'
    
    namef=0
    namenf=0
    namef_playlist_names_list=[]
    namenf_playlist_names_list=[]
    # for playlist, playlistn in zip(paylist_path_list, paylist_name_list):
    for playlist, playlistn in zip(paylist_path_list[:num_playlists], paylist_name_list[:num_playlists]):
        print(f"\n\n\nplaylist: {playlistn}")
        
        if playlistn not in playlist_2_prof_name_dict.keys():
            print(f"{playlistn} not in playlist_2_prof_name_dict")
            continue
            
        true_name = playlist_2_prof_name_dict[playlistn]
        if true_name is None:
            print(f"no name for {playlistn}")
            continue
            
        if true_name not in folder_name_2_prof_name_dict.values():
            print(f"no face for {true_name}")
            continue
        
        results_playlist_dir = os.path.join(results_chanel_dir, playlistn)
        
        vedio_path_list = get_extension_files(playlist, ext='.mp4')
        vedio_name_list = [x.split('\\')[-1] for x in vedio_path_list] 
        
        for vedio_path in vedio_path_list[vedio_start_idx:vedio_start_idx+num_vedios]:
            vname = vedio_path.split('\\')[-1]

            video_capture = cv2.VideoCapture(vedio_path)

            vst = 20
            vet = 5*60
            num_frames_per_sec = 1

            fps = int(video_capture.get(cv2.CAP_PROP_FPS))
            st = tt()
            i=-1
            pred_cnt = 0
            y_pred_list = {model:[] for model in serengil_deep_face_models_list}
            names_list_list = {model:[] for model in serengil_deep_face_models_list}
            while video_capture.isOpened():
                i+=1
                if int(i/fps) < vst:
                    ret, frame = video_capture.read()
                    continue

                if int(i/fps) > vet:
                    break

                 # Grab a single frame of video
                ret, frame = video_capture.read()

                # process
                if i%int(fps/num_frames_per_sec)==0:
                    # TODO: PLS find a workaround this is too stupid
                    cv2.imwrite(temp_img_path, frame)
                    try:
                        detected_face = DeepFace.detectFace(temp_img_path, detector_backend = best_backend)
                        pred_cnt+=1
                    except ValueError:
#                     except Exception as e:
#                         print(e)
                        continue
                    
                    if pred_cnt>max_pred_cnt:
                        break
                        
                    # for every model, for every labeled img folder, 
                    # avearge distance from labeled and detected img
                    for j, model in enumerate(serengil_deep_face_models_list):
                        # TODO: find a way to not print warnings or tqdms
                        df = stopPrint(DeepFace.find, 
                                           img_path = temp_img_path, 
                                           detector_backend = best_backend,
                                           model_name = model,
                                           distance_metric = "euclidean",
                                           enforce_detection = False,
                                           db_path = sgdf_known_faces_dir)
                        
                        df['identity'] = df['identity'].apply(change_img_name)
                        df = df.groupby('identity', as_index=False)['distance'].mean()
                        if df.empty:
                            y_pred_list[j].append(0)
                            continue
                            
                        optim_folder_name = df.sort_values('identity').iloc[0]['identity']
                        name = folder_name_2_prof_name_dict[optim_folder_name]
                        names_list_dict[model].append(name)
                        if name==true_name:
                            y_pred_dict[model].append(1)
                        else:
                            y_pred_dict[model].append(0)

                # Hit 'q' on the keyboard to quit!
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            # Release handle to the webcam
            video_capture.release()
            cv2.destroyAllWindows()
            
            print(f"\n\nvedio: {vname}")
            print(f"true_name: {true_name}")
            for j, model in enumerate(serengil_deep_face_models_list):
                y_pred = y_pred_dict[model]
                names_list = names_list_dict[model]
                print(f"\nunique predicted value counts for model: {model}:")
                unique, counts = np.unique(np.array(names_list), return_counts=True)
                print(np.asarray((unique, counts)).T)
                print(f"\naccuracy:{np.array(y_pred).mean()} \
                \nnumber of predictions: {len(y_pred)}")
                
            mean_y_pred_list = list(np.array(y_pred_list).mean(axis=1))
            best_model_idx = mean_y_pred_list.index(max(mean_y_pred_list))
            best_model = serengil_deep_face_models_list[best_model_idx]
            best_model_list.append(best_model)
            best_acc = mean_y_pred_list[best_model_idx]
            print(f"\nbest model: {model} \n accuracy:{best_acc} \n time: {tt()-st}")

print("unique best model contender counts:")
unique, counts = np.unique(np.array(best_model_list), return_counts=True)
print(np.asarray((unique, counts)).T)
best_model = max(best_model_list, key=best_model_list.count)
print(f"\nbest of the best model: {best_model}")

winsound.Beep(freq, duration)





for nptel_channel_name Biotechnology



playlist: Bio electricity



Analyzing:   0%|                                                                                 | 0/1 [00:00<?, ?it/s]

Analyzing: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.71s/it]


Analyzing: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.06s/it]




vedio: QiwxdcckPGc.mp4
true_name: prof. mainak das

unique predicted value counts for model: VGG-Face:
[['prof. shalabh ' '5']]

accuracy:0.0                 
number of predictions: 5

unique predicted value counts for model: Facenet:
[['prof. mainak das' '1']
 ['prof. shalabh ' '4']]

accuracy:0.2                 
number of predictions: 5

unique predicted value counts for model: OpenFace:
[['dr eue or' '1']
 ['prof: padma vankar' '1']]

accuracy:0.0                 
number of predictions: 5

unique predicted value counts for model: DeepFace:
[['prof. amit basak ' '1']
 ['prof. indranil sengupta' '1']
 ['prof. mainak das' '1']
 ['prof. soumya k. ghosh' '1']
 ['prof: padma vankar' '1']]

accuracy:0.2                 
number of predictions: 5

unique predicted value counts for model: DeepID:
[]

accuracy:0.0                 
number of predictions: 5

unique predicted value counts for model: Dlib:
[['prof. shalabh ' '5']]

accuracy:0.0                 
number of predictions: 5

best mo

Analyzing: 100%|█████████████████████████████████████████████████████████████████████████| 1/1 [00:51<00:00, 51.89s/it]


In [ ]:
# mean_distance_list = []
# mean_vote_list = []
# for img_path_list in known_face_img_path_nested_list:
#     vote_list = []
#     distance_list = []
#     for img_path in img_path_list[:num_labeled_faces]:
#         result  = DeepFace.verify(img_path, temp_img_path, 
#                                   model_name = model, 
#                                   distance_metric = deep_face_metric,
#                                   detector_backend = best_backend,
#                                   enforce_detection =False)
#         vote_list.append(result["verified"])
#         distance_list.append(result["distance"])
#     majority_vote = max(vote_list, key=vote_list.count)
#     mean_vote_list.append(len[w for w in vote_list if w])
#     mean_distance_list.append(np.array(distance_list).mean())

# #                         optim_dist_idx =  mean_vote_list.index(max(mean_vote_list))
# optim_dist_idx =  mean_distance_list.index(min(mean_distance_list))

In [ ]:
# prof_name_2_playlist_dict = {'prof. t.k.sengupta': 'Foundation of Scientific Computing', 
#  None: 'Particle Characterization (PG)',
#  'prof. k. p. sinhamahapatra': 'Introduction to Aerodynamics',
#  'dr. a. kushari': 'Jet and Rocket Propulsion',
#  'prof. bhaskar ot': 'Turbomachinery Aerodynamics',
#  'prof. mainak das': 'Bio electricity',
#  'dr. p. gopinath ': 'Biomedical nanotechnology',
#  'prof. debabrata das': 'Industrial Biotechnology',
#  'prof. shamik sen': 'Introduction to Mechanobiology',
#  'dr. ton er': 'Introduction To Proteomics',
#  'prof s. de': 'Advanced Mathematical Techniques in Chemical Engineering',
#  'dr eue or': 'Mass Transfer Operations I',
#  'prof. s. ganguly': 'Microscale Transport Processes',
#  'prof. gargi das epartment': 'Phase Equilibrium Thermodynamics',
#  'prof: padma vankar': 'Advance Analytical Course',
#  'prof. debashis ray ': 'Analytical Chemistry',
#  'prof. s. dasgupta coma': 'BioChemistry I',
#  'prof. a. g. samuelson': 'Introduction to Organometallic Chemistry',
#  'prof. amit basak ': 'Stereochemistry',
#  'prof. deepak khemani': 'Artificial Intelligence Search Methods for problem Solving',
#  'prof. soumya k. ghosh': 'Cloud Computing',
#  'prof. indranil sengupta': 'Hardware Modeling using Verilog',
#  'dr. yogish sabharwal': 'Introduction to parallel Programming in Open MP',
#  'prof. shalabh ': 'Introduction to R Software'}

# playlist_2_prof_name_dict = {v: k for k, v in prof_name_2_playlist_dict.items()}
# save_as_pickle(os.path.join(pkl_files_dir, "prof_name_2_playlist_dict.pkl"), prof_name_2_playlist_dict)
# save_as_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"), playlist_2_prof_name_dict)


# folder_name_2_prof_name_dict = {0: 'prof. k. p. sinhamahapatra',
#  1: 'dr. a. kushari',
#  2: 'prof. bhaskar ot',
#  3: 'prof. mainak das',
#  4: 'dr. p. gopinath ',
#  5: 'prof. debabrata das',
#  6: 'prof. shamik sen',
#  7: 'dr. ton er',
#  8: 'prof s. de',
#  9: 'dr eue or',
#  10: 'prof. s. ganguly',
#  11: 'prof. gargi das epartment',
#  12: 'prof: padma vankar',
#  13: 'prof. debashis ray ',
#  14: 'prof. s. dasgupta coma',
#  15: 'prof. a. g. samuelson',
#  16: 'prof. amit basak ',
#  17: 'prof. deepak khemani',
#  18: 'prof. soumya k. ghosh',
#  19: 'prof. indranil sengupta',
#  20: 'prof. shalabh '}

# prof_name_2_folder_name_dict = {v: k for k, v in folder_name_2_prof_name_dict.items()}
# save_as_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict.pkl"), folder_name_2_prof_name_dict)
# save_as_pickle(os.path.join(pkl_files_dir, "prof_name_2_folder_name_dict.pkl"), prof_name_2_folder_name_dict)